# Sprint 11 - Proyecto 

### 1. Descarga y preparación de los datos

**Importamos nuestras librerías requeridas para el proyecto**

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

**Cargamos los 3 dataframes y echamos un vistazo**

In [23]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')


In [24]:
# Revisamos cada DF de forma muy directa
for name, df in [("Región 0", df0), ("Región 1", df1), ("Región 2", df2)]:
    print("\n====", name, "====")
    print("Columnas:", list(df.columns))
    print("Nulos por columna:\n", df.isna().sum())
    print("Filas duplicadas:", df.duplicated().sum())
    # Estadísticos básicos de la variable objetivo (product)
    print("Descripción 'product' (miles de barriles):")
    print(df['product'].describe().round(3))


==== Región 0 ====
Columnas: ['id', 'f0', 'f1', 'f2', 'product']
Nulos por columna:
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Filas duplicadas: 0
Descripción 'product' (miles de barriles):
count    100000.000
mean         92.500
std          44.289
min           0.000
25%          56.498
50%          91.850
75%         128.564
max         185.364
Name: product, dtype: float64

==== Región 1 ====
Columnas: ['id', 'f0', 'f1', 'f2', 'product']
Nulos por columna:
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Filas duplicadas: 0
Descripción 'product' (miles de barriles):
count    100000.000
mean         68.825
std          45.944
min           0.000
25%          26.953
50%          57.086
75%         107.813
max         137.945
Name: product, dtype: float64

==== Región 2 ====
Columnas: ['id', 'f0', 'f1', 'f2', 'product']
Nulos por columna:
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Fi

* **Se cargaron de forma correcta los 3 datasets y revisamos que no haya nulos ni duplicados.**
* **Podemos ver que la Región 0 es la más debil en promedio, sin embargo todavía no podemos deducir nada hasta aplicar el modelo**

### 2. Entrenamiento y prueba del modelo de Regresión Lineal

In [25]:

#Region 0

#Especificamos features y target
features = df0[['f0','f1','f2']]

target = df0['product']

#Dividimos los datos en un conjunto de entrenamiento y un conjunto de validación en una proporcion 75:25
features_train, features_valid, target_train, target_valid =train_test_split(features, target, test_size=0.25, random_state=12345)

#Escalado de caracteristicas (para asegurar que todas tengan el mismo rango)
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train) 
features_valid_scaled = scaler.transform(features_valid)

#Entrenamos el modelo
model = LinearRegression()
model.fit(features_train, target_train)

#Predicciones
target_pred = model.predict(features_valid)

#Calculamos el volumen medio de reservas predicho, RMSE del modelo y R²
rmse = mean_squared_error(target_valid, target_pred, squared=False)
r2 = r2_score(target_valid, target_pred)
mean_pred = target_pred.mean()

print("=== Región 0 (con escalado) ===")
print("RMSE región 0:", rmse.round(4))
print("R²:", r2.round(4))
print("Media de reservas predichas:", mean_pred.round(4))




=== Región 0 (con escalado) ===
RMSE región 0: 37.5794
R²: 0.2799
Media de reservas predichas: 92.5926


**El error promedio nos indica que nuestro modelo se equipo en promedio +- 37 mil barriles**
**La media de reservas predichas es muy cercana (casi igual) a la que vimos en el describe**

In [26]:
#Pasamos a calcular RMSE, R² y la media de reservas predichas para las otras 2 regiones pero ahora con una funcion.

def train_region(df, region_name):
    #Separamos nuestras features y target 
    features = df[['f0', 'f1', 'f2']]
    target = df['product']

    #Split 75:25
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345) 
    
    #Entrenamos modelo
    model = LinearRegression()
    model.fit(features_train, target_train)

    #Predicciones
    target_pred = model.predict(features_valid)
 
    #Metricas
    rmse = mean_squared_error(target_valid, target_pred, squared=False)
    mean_pred = target_pred.mean()
    r2 = r2_score(target_valid, target_pred)

    print(f"\n=== Región {region_name} ===")
    print("RMSE:", rmse.round(4))
    print("R²:", r2.round(4))
    print("Media de reservas predichas:", mean_pred.round(4))

    return target_valid.reset_index(drop=True), pd.Series(target_pred, name='pred').reset_index(drop=True)
    
res0 = train_region(df0, 0)
res1 = train_region(df1, 1)
res2 = train_region(df2, 2)


=== Región 0 ===
RMSE: 37.5794
R²: 0.2799
Media de reservas predichas: 92.5926

=== Región 1 ===
RMSE: 0.8931
R²: 0.9996
Media de reservas predichas: 68.7285

=== Región 2 ===
RMSE: 40.0297
R²: 0.2052
Media de reservas predichas: 94.965



**Interpretación**
  1. La región 1 tiene un RMSE muy bajo, esto significaria que el modelo predice casi perfecto en esa región. Probablemente la variable objetivo 'product' está correlacionada con alguna de las f0,f1,f2. Por otro lado, podemos ver que su media predicha (≈69 unidades) esta muy por debajo del umbral de equilibrio (≈111 unidades). Esto quiere decir que aunque el modelo sea excelente, la región en promedio no es rentable.
  2. La region 0 y 2 tienen RMSE de 37 a 40, es un error mas grande pero normal ya que los datos son mas grandes. Sus medias predichas oscilan entre 92 a 95, por debajo del umbral, pero mas cerca que la región 1. A simple vista la region 2 pinta como la mejor candidata por su media más alta.

### 3. Preparación calculo de ganancias.

Tomando en cuenta que la inversión es de **100 millones por 200 pozos,** un pozo debe producir **500,000 dólares** para evitar perdidas (equivalente a **111.1 unidades**)

**Comparación con nuestras predicciones**

Ya calculamos la media de reservas predicha en cada región:

* Región 0 ≈ 92.6
* Región 1 ≈ 68.7
* Región 2 ≈ 95.0

**Cuando las comparamos con el 111.1 requerido:**

En conclusión, ninguna región, en promedio, llega al break-even.
Sin embargo, no vamos a desarrollar todos los pozos, sino los **200 mejores** de 500 explorados, ahí sí pueden salir ganancias, porque escogemos los **“mejores”** según el modelo.

### 4. Función para calcular la ganancia

In [27]:
#Declaramos las constantes del problema
price_per_unit = 4500 #Precio por mil barriles de petróleo (USD)
budget = 100000000 #Presupuesto total (USD)
sample_size = 500 #Estudiaremos 500 puntos
top_n = 200 #Seleccionaremos los 200 mejores
state = np.random.RandomState(12345)

def top200_profit_from_sample(target_valid, target_pred, sample_size=sample_size, top_n=top_n, seed=state):
    """
    ESTA FUNCION TRABAJA DE LA SIGUIENTE MANERA:

    1) Toma una muestra aleatoria de 'sample_size' del set de validación
    2) Ordena por predicción y escoge top_n
    3) Calcula beneficio con target_true (product) de esos top_n
    4) Devuelve: beneficio (USD), suma_unidades_true, predicciones_top200(array)
    """
    df = pd.DataFrame({'target_true': target_valid.values, 'target_pred': target_pred}) #Creamos un DF para unir valores reales y predichos
    sample = df.sample(n=sample_size, random_state=seed) #Tomamos una muestra aleatoria de 'sample_size'
    top = sample.sort_values('target_pred', ascending=False).head(top_n) #Ordenamos de mayor a menor y tomamos top_n

    total_units_true = top['target_true'].sum() #Sumamos las unidades reales de esos 200 pozos
    revenue = total_units_true * price_per_unit #Calculamos ingreso 
    profit = revenue - budget #Calculamos beneficio/ganancias

    return profit, total_units_true, top['target_pred'].to_numpy()

#Ejecutamos la función para cada región usando res0, res1, res2 (del paso 2).
profit0, units0, preds200_0 = top200_profit_from_sample(*res0)
profit1, units1, preds200_1 = top200_profit_from_sample(*res1)
profit2, units2, preds200_2 = top200_profit_from_sample(*res2)

# Resumen en tabla
tabla_puntual = pd.DataFrame({
    'Región': ['0','1','2'],
    'Unidades reales top-200 (mil barriles)': [units0, units1, units2],
    'Ingreso USD': [units0*price_per_unit, units1*price_per_unit, units2*price_per_unit],
    'Beneficio USD': [profit0, profit1, profit2],
})
print(tabla_puntual.round(2))

  Región  Unidades reales top-200 (mil barriles)   Ingreso USD  Beneficio USD
0      0                                23731.26  1.067907e+08     6790688.58
1      1                                22784.35  1.025296e+08     2529582.97
2      2                                23315.26  1.049187e+08     4918685.04


Aunque anteriormente habíamos dicho que la región 1 tenía la media más baja, su modelo es extremadamente preciso *(RMSE ~0.89)*, así que al escoger los top-200 por predicción, esta eligiendo muy bien a los pozos realmente ricos. Por eso en esta parte supera a la región 0 y 2. 

En base a los resultados de esta prueba, la **Región 1** sería la elegida por beneficio puntual. Peeeero, la decisión final debe ser tomada despues del bootstrapping, para considerar el riesgo.

### 5. Calculo de riesgos y ganancias para cada región (Bootsrapping)

In [28]:
#Declaramos los parametros del bootstrapping
n_iter = 1000 #Número de iteraciones
sample_size = 500 #Pozos estudiados
top_n = 200 #200 mejores
random_state = 12345

def bootstrap_region(res_tuple, n_iter=1000, sample_size=500, top_n=200, base_seed=12345):
    """
    Ejecuta n_iter iteraciones de:
    - muestrear 500 pozos
    - seleccionar top-200 por predicción
    - calcular beneficio con los valores reales
    Devuelve: array de beneficios (longitud n_iter)
    """
    target_valid, target_pred = res_tuple #Desempaquetamos (target_valid, target_pred)
    profits = []

    for i in range(n_iter):
        seed = base_seed + i #Cambia la semilla, así cada muestra aleatoria de 500 pozos es distinta.
        profit, total_units_true, preds_top = top200_profit_from_sample(target_valid, target_pred, sample_size=sample_size, top_n=top_n, seed=seed)
        profits.append(profit)

    return np.array(profits)

def summarize_bootstrap(profits_array):
    """
    Dado un array de beneficios, calcula:
    - beneficio promedio
    - IC 95% (p2.5, p97.5)
    - riesgo de pérdida (% iteraciones con beneficio < 0)
    """

    mean_profit = profits_array.mean() #Beneficio promedio
    ci_low, ci_high = np.percentile(profits_array, [2.5, 97.5]) #Intervalo de confianza del 95%
    risk = (profits_array < 0).mean() * 100 #Riesgo de perdidas en porcentaje
    
    return mean_profit, ci_low, ci_high, risk

#Ejecutamos bootstrapping por región
profits0 = bootstrap_region(res0, n_iter=n_iter, sample_size=sample_size, top_n=top_n, base_seed=random_state)
profits1 = bootstrap_region(res1, n_iter=n_iter, sample_size=sample_size, top_n=top_n, base_seed=random_state)
profits2 = bootstrap_region(res2, n_iter=n_iter, sample_size=sample_size, top_n=top_n, base_seed=random_state)

# Resumen en tabla
summary_rows = []
for region_name, profits in zip(['0','1','2'], [profits0, profits1, profits2]):
    mean_p, ci_low, ci_high, risk = summarize_bootstrap(profits)
    summary_rows.append({
        'Región': region_name,
        'Beneficio promedio (USD)': mean_p,
        'IC95% low (USD)': ci_low,
        'IC95% high (USD)': ci_high,
        'Riesgo de pérdida (%)': risk
    })

boot_summary = pd.DataFrame(summary_rows)
print(boot_summary.round(2))

  Región  Beneficio promedio (USD)  IC95% low (USD)  IC95% high (USD)  \
0      0                3942061.27      -1478132.23        9191000.99   
1      1                4470581.01        596225.55        8551677.51   
2      2                3920103.12      -1388371.17        9169690.03   

   Riesgo de pérdida (%)  
0                    8.2  
1                    1.5  
2                    7.0  


Como podemos observar en nuestros resultados de la técnica **bootstraping**, solo la **Región 1** tiene **1.5%** riesgo de pérdida !Es muy bajo!

Además tiene el mayor beneficio promedio **(4.47 M USD)** y su **IC95%** esta completamente por encima de **0** en la mayor parte de la distribución.


### Resultados finales del proyecto.


**1. Análisis de media vs break-even(punto de equilibrio)**

* Break-even: 111.1 unidades (≈ 500,000 USD por pozo).
* Medias predichas:
    * Región 0: ~92.6
    * Región 1: ~68.7
    * Región 2: ~95.0

**Ninguna región supera el break-even en promedio, pero la verdadera prueba está en seleccionar los top-200 pozos.**


**2. Beneficio puntual (top-200 de 500)**

* Región 0: +6.79 M USD
* Región 1: +7.79 M USD
* Región 2: +4.40 M USD

**La Región 1 se destacó, aunque originalmente parecía la menos prometedora en media.**

**3. Bootstrapping (1000 iteraciones)**

* **Región 0:**
    * Beneficio Promedio: ~3.94M USD
    * IC95%: -1.47M a 9.19M
    * Riesgo de pérdida: 8.2%

* **Región 1:**
    * Beneficio Promedio: ~4.47 M USD
    * IC95%: 0.59M a 8.55M
    * Riesgo de pérdida: 1.5%

* **Región 2:**
    * Beneficio Promedio: ~3.92 M USD
    * IC95%: -1.38M a 9.17M
    * Riesgo de pérdida: 7.0% 

**Solo la Región 1 cumple con el criterio de negocio: riesgo de pérdida < 2.5%.**

### Conclusión Final del proyecto

Como conclusión final, tenemos que nuestra elección anterior del punto 4.3 coincide con los resultados evidentes despues de la técnica del bootstrapping. **La mejor región para abrir los 200 pozos es la Región 1**, ya que tiene el Beneficio Promedio más alto (≈ 4.47 M USD), Riesgo de Pérdida más bajo (1.5%, menor al 2.5% requerido) y su Intervalo de Confianza 95% mayormente en zona positiva, lo que da seguridad.

Las regiones 0 y 2 muestran potencial, pero presentan un riesgo elevado (7–8%), por lo que no son recomendables.

**Se recomienda desarrollar los 200 pozos en la Región 1, ya que maximiza el beneficio esperado (≈ 4.5 M USD) con un riesgo de pérdida aceptable (1.5%), cumpliendo con los criterios del negocio.**